In [4]:
import kagglehub
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

import random

## 1. Download dataset from Kaggle


In [16]:
# This gets saved locally on your machine, check the path for the location
path = kagglehub.dataset_download("tarundalal/anime-quotes")
print("Path to dataset files:", path)


Path to dataset files: /Users/maxfrischknecht/.cache/kagglehub/datasets/tarundalal/anime-quotes/versions/1


## 2. Load the dataset with a given filename


In [17]:
filename = "AnimeQuotes.csv"  # Specify the filename here
dataset_path = os.path.join(path, filename)

if os.path.exists(dataset_path):
    df = pd.read_csv(dataset_path)
    print("Dataset loaded successfully:")
    print(df.head())
else:
    raise FileNotFoundError(f"File {filename} not found in the dataset directory.")


Dataset loaded successfully:
                                               Quote         Character  \
0  People’s lives don’t end when they die, it end...     Itachi Uchiha   
1  If you don’t take risks, you can’t create a fu...    Monkey D Luffy   
2   If you don’t like your destiny, don’t accept it.    Naruto Uzumaki   
3       When you give up, that’s when the game ends.  Mitsuyoshi Anzai   
4  All we can do is live until the day we die. Co...      Deneil Young   

                             Anime  
0                           Naruto  
1                        One Piece  
2                           Naruto  
3                        Slam Dunk  
4  Uchuu Kyoudai or Space Brothers  


## 3. Extract quotes and preprocess text


In [18]:
# Extract the "Quote" column from the DataFrame
quotes = df['Quote']

# Convert all values in the column to strings (in case they are not already)
quotes = quotes.astype(str)

# Convert the column into a Python list (so it can be processed further)
quotes = quotes.tolist()

# quotes

## 4. Tokenize the text data


In [19]:
# Create a tokenizer object to process the text
# This object will be used to convert words into numerical values (tokens).
tokenizer = Tokenizer() # num_words=5000 limit to improve model based on low learning performance

# Fit the tokenizer on the list of quotes
# This builds a vocabulary based on the words in the quotes dataset
# Counts the frequency of each word in the dataset.
# Assigns a unique index to each word.
tokenizer.fit_on_texts(quotes)

# Converts each quote into a sequence of numbers based on the learned word indices.
sequences = tokenizer.texts_to_sequences(quotes)
sequences[:1]

[[306, 307, 18, 101, 28, 32, 61, 5, 141, 28, 32, 55, 308]]

## 5. Create input-output pairs for training


In machine learning, especially in sequence models, we train a model using input-output pairs where:
- The input is a partial sequence of words.
- The output is the next word that the model should predict.
- This code creates input-output pairs for training the text generation model using n-gram-like sequences.
- It progressively increases the sequence length so that the model learns contextual relationships between words.
- It helps the model predict the next word in a sequence.


```py
[10, 15]            # "The power"
[10, 15, 22]        # "The power is"
[10, 15, 22, 8]     # "The power is strong"
```

In [21]:
# Initialize an empty list to store input sequences
input_sequences = []

# Loop through each tokenized sequence (each quote in numerical format)
for seq in sequences:
    # Generate progressively growing sub-sequences from each full sequence
    for i in range(1, len(seq)):  
        # Append a sub-sequence that starts from the first word up to the i-th word
        input_sequences.append(seq[:i+1])

# input_sequences

# 6. Pad sequences and define input & output

- Loops through all input_sequences and finds the longest sequence. (deep learning models, require fixed-size inputs.)
- Some sequences are shorter, so we pad them with zeros to match the longest sequence. (Ensures alignment for LSTMs, which process sequences from left to right. The actual words stay at the end, and LSTMs focus on recent words.)
- We then split them into `x_train_input` which are the word sequences minus the last word (which get saved into `y_train_input`) to learn to predict the next word

| **X (Input Sequence)**  | **y (Label - Next Word)** |
|-------------------------|--------------------------|
| `[10]`                 | `15` ("power")           |
| `[10, 15]`             | `22` ("is")             |
| `[10, 15, 22]`         | `8` ("strong")          |

- Convert y labels (word indices) into *one-hot encoded vectors*. This is necessary because the output layer of the model expects a probability distribution

```py
# from
y = [3, 7, 1, 9]
# to (if there would be 10 possible words)
[
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # Word ID 3
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],  # Word ID 7
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Word ID 1
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   # Word ID 9
]
```

In [22]:
# Find the longest sequence length in input_sequences
# This ensures all sequences will be padded to the same maximum length
max_sequence_length = max([len(seq) for seq in input_sequences]) # Increase length,  Increase the sequence length so the model sees more context.

# Pad all sequences to the maximum length found
# 'pre' padding means zeros are added at the beginning of shorter sequences
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# In sequence models, we train the model to predict the next word in a sequence. This means:
# X (features): The input words the model sees. / y (labels): The next word the model should predict.
# Extract input sequences (all words except the last one)
x_train_input = input_sequences[:, :-1]  
# Extract target labels (the last word in each sequence)
y_train_labels = input_sequences[:, -1]  

# Convert y labels (word indices) into one-hot encoded vectors
# This is necessary because the output layer of the model expects a probability distribution
y_train_labels = tf.keras.utils.to_categorical(y_train_labels, num_classes=len(tokenizer.word_index) + 1)

# 7. Define the LSTM Model


**Embedding**
- The Embedding layer transforms word indices (integers) into dense vector representations.
- `input_dim`: Number of unique words (vocabulary size) +1 for padding. Padding are the `0` added to make all sentences the same length. Since we used padding tokens (0), the model must know how to handle 0 as a special token.
- `output_dim`: Size of the dense vector that represents each word. The higher the dimension, the more information can be captured about each word.
- `input_length`: This is the length of the input sequence to be used to train (longest - 1)

**LSTM**
- 256 memory units (neurons) that store past information and learn dependencies between words.
- Ensures the LSTM outputs the entire sequence, instead of just the last output.


In [23]:
model = Sequential([

    # Embedding layer: Converts word indices into dense vector representations
    # input_dim = vocabulary size (+1 for padding index)
    # output_dim = 128 (size of each word vector)
    # input_length = max_sequence_length - 1 (because we predict the next word)
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length-1),

    # First LSTM layer: Processes sequential data
    # 256 units = number of memory cells (neurons)
    # Maybe 256 is to large is the data set is small > 128
    # return_sequences=True allows the next LSTM layer to receive the full sequence output
    LSTM(256, return_sequences=True),

    # Second LSTM layer: Learns deeper sequence patterns
    # 256 units (neurons)
    # Since this is the last LSTM layer, return_sequences=False (default)
    LSTM(256), # Further reduce complexity

    #Dropout(0.3),  # Another Dropout layer

    # Fully connected (Dense) output layer:
    # Number of neurons = vocabulary size (+1 for padding index)
    # Softmax activation converts logits into probability distribution over words
    Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

/Users/maxfrischknecht/.pyenv/versions/3.11.5/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
# Define the loss function separately
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Compile the model using the separate loss function
# Adam works well for general tasks, but RMSprop is better for sequential tasks like NLP
# # Use RMSprop optimizer (better for sequential data)
# optimizer=RMSprop(learning_rate=0.0003),
# Adam(learning_rate=0.0005)
model.compile(
            loss=loss_fn, 
            optimizer='adam',
            metrics=['accuracy'])

## 8. Train the Model


- `x_train_input`: The input sequences (X) for training
- `y_train_labels`: The corresponding one-hot encoded labels (y)
- `epochs=30`: The model will go through the entire dataset 30 times during training. More epochs mean the model learns more, but too many can lead to overfitting.
- `batch_size=64`: Use mini-batches of 64 samples for each training step. Instead of training on one sample at a time (slow) or all samples at once (high memory usage), the model is trained in mini-batches.

**Notes on Batch Sizes**

🔹 When Does a Bigger Batch Size Help?
- If you have a lot of GPU memory → Larger batches train faster.
- If you need stable weight updates → Large batches reduce update noise.
- For well-defined problems (e.g., image classification) → Large batches work well.

🔹 When Should You Use a Smaller Batch Size?
- For NLP models (like LSTMs, Transformers) → Small batches generalize better.
- If you have limited memory → Small batches fit in memory.
- If your dataset is small → Small batches prevent overfitting.

🔹 What’s the Best Batch Size?
- There’s no universal best batch size—it depends on your dataset and hardware.

🔹 Common recommendations:
- 32 or 64 → Good for most NLP models.
- 128 or 256 → Good for CNNs on large datasets.
- 512+ → Only useful if you have a huge dataset & powerful GPU.

🔹 Try different values and compare accuracy/loss.
- You can use a validation set to find the best batch size.

| **Metric**            | **Ideal Value**      | **Your Value**  | **Fix Needed?**      |
|----------------------|--------------------|----------------|--------------------|
| **Training Accuracy**  | ✅ High (~95-98%)   | **97.22%**     | ❌ No issue         |
| **Validation Accuracy**| ✅ Similar to training (~90%) | **17.41%** | ✅ **Yes (overfitting)** |
| **Training Loss**     | ✅ Low (~0.1 or less) | **0.0729**     | ❌ No issue         |
| **Validation Loss**   | ✅ Low (~close to training) | **4.9760**  | ✅ **Yes (big gap)** |

In [26]:
# use early stopping to prevent overfitting when training with more epochs
# If val_loss does not improve for patience=5 consecutive epochs, training stops.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Since NLP models often work better with larger batches: Larger batch size stabilizes updates and avoids too much noise.
history = model.fit(x_train_input, y_train_labels, epochs=30, batch_size=64)

Epoch 1/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 226ms/step - accuracy: 0.0485 - loss: 5.6511
Epoch 2/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 236ms/step - accuracy: 0.0605 - loss: 5.5451
Epoch 3/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 245ms/step - accuracy: 0.0528 - loss: 5.6249
Epoch 4/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 227ms/step - accuracy: 0.0593 - loss: 5.5523
Epoch 5/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 211ms/step - accuracy: 0.0553 - loss: 5.4639
Epoch 6/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 244ms/step - accuracy: 0.0653 - loss: 5.3312
Epoch 7/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 258ms/step - accuracy: 0.0647 - loss: 5.2943
Epoch 8/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 231ms/step - accuracy: 0.0767 - loss: 5.1514
Epoch 9/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 224ms/step - accuracy: 0.0913 - loss: 5.0058
Epoch 10/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 209ms/step - accuracy: 0.1127 - loss: 4.8481
Epoch 11/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 232ms/step - accuracy: 0.1234 - loss: 4.6608
Epoch 12/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 224ms/step

In [145]:
# save the model manually for later contiuing
# model.save("anime_quote_generator_manual_checkpoint.h5")
model.save("anime_quote_generator.keras")
print("Checkpoint saved!")

Checkpoint saved!


We can visualize overfitting by plotting the accuracy and loss.

In [27]:
# Extract training history
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_acc) + 1)

# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs. Validation Accuracy')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs. Validation Loss')

plt.show()

KeyError: 'val_accuracy'

## 9. Generate New Quotes



Generates a new quote by predicting the next words based on seed_text.

Parameters:
- seed_text (str): The initial text to start generating from.
- next_words (int): Number of words to generate.

Returns: 
- str: The generated quote.


In [28]:
# Step 9: Generate New Quotes
def generate_quote(seed_text, next_words=20):
    # Loop to generate 'next_words' number of words
    for _ in range(next_words):
        
        # Convert the input text into a sequence of token indices
        tokenized = tokenizer.texts_to_sequences([seed_text])[0]
        
        # Pad the sequence to match the expected input size of the model
        # Padding ensures the sequence is of the same length as used during training
        tokenized = pad_sequences([tokenized], maxlen=max_sequence_length-1, padding='pre')
        
        # Predict the next word using the trained model
        # np.argmax() gets the index of the highest probability word
        predicted = np.argmax(model.predict(tokenized, verbose=0), axis=-1)
        
        # Convert the predicted word index back to an actual word
        # If the word is not found, return an empty string to avoid errors
        word = tokenizer.index_word.get(predicted[0], '')

        # Append the predicted word to the seed text
        seed_text += " " + word

    # Return the final generated quote
    return seed_text

**Generate a random new quote**
- Select a random seed quote to give the model context to start from. 
- This is not necessary but encouraged as the model might make unpredictable things
    1.	If the words in seed_text exist in the vocabulary, the model treats it like any other input.
	2.	If the words are not in the vocabulary, they will be ignored or tokenized as Unknown (UNK).

In [39]:
random_quote = random.choice(quotes[:50])
# start_seed = random_quote
print("Selected Quote:")
print(f"{random_quote} \n")

print("Generated Quote:")
print(generate_quote(random_quote, next_words=10))

Selected Quote:
I’ll leave tomorrow’s problems to tomorrow’s me. 

Generated Quote:
I’ll leave tomorrow’s problems to tomorrow’s me. them to do anything always something for i have the
